In [1]:
import pandas as pd
from pandas.io import sql
import sqlite3
import re

In [2]:
conn = sqlite3.connect('ArticlesScraping.db')

sql.read_sql(
"""
select * from ArticleCorpus
""", con=conn).head(5)

,PageLinks,FullLink,Time,Author,Header,Body,CleanedBody
0,/wealth-manager/news/saturday-papers-rbs-to-ch...,http://citywire.co.uk/wealth-manager/news/satu...,2016-08-20 05:17:37,>HimanshuSingh],Saturday Papers: RBS to charge big clients for...,"[<div class=""article-body"" itemprop=""articleBo...",[\nTop stories\n\nFinancial Times: Royal Bank ...
1,/wealth-manager/news/rathbones-adds-to-london-...,http://citywire.co.uk/wealth-manager/news/rath...,2016-08-19 13:42:56,>JoshuaThurston],Rathbones adds to London office with second HS...,"[<div class=""article-body"" itemprop=""articleBo...",[\nRathbone Brothers has hired its second HSBC...
2,/wealth-manager/news/central-banks-pouring-fue...,http://citywire.co.uk/wealth-manager/news/cent...,2016-08-19 12:52:45,>AndrewMilligan],Central banks pouring fuel on dying embers,[],[]
3,/wealth-manager/news/wealth-manager-olympics-t...,http://citywire.co.uk/wealth-manager/news/weal...,2016-08-19 12:27:35,[],Wealth Manager Olympics: the Relay,"[<div class=""article-body jsArticle-body--star...","[\nIf like us, the Wealth Manager Olympics has..."
4,/wealth-manager/news/jupiters-davies-buys-bomb...,http://citywire.co.uk/wealth-manager/news/jupi...,2016-08-19 11:21:00,>JonathanYarker],Jupiter's Davies buys 'bombed out' domestics,"[<div class=""article-body"" itemprop=""articleBo...","[\n\xa0Steve Davies, manager of the\xa0Jupiter..."


In [3]:
#Need to add a line of code here to remove duplicates in table (based on url? Header? time?)

In [4]:
#Need to add a line of code here to clean up the Author column

In [5]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
# Make a list of the words you want remove# Remove stop words

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

# Make it a raw string
def raw_string(s):
    if isinstance(s, str):
        s = s.encode('string-escape')
    elif isinstance(s, unicode):
        s = s.encode('unicode-escape')
    return s

# Remove stop words
def remove_stopwords(content):
    cleaned = filter(lambda x: x not in ENGLISH_STOP_WORDS,content.split())
    return ' '.join(cleaned)

#remove punctuation
import string
def remove_punctuation(content):
    return filter(lambda x: x in string.ascii_letters+" ",content)

#remove numbers
def remove_numbers(content):
    return re.sub(r'\d+', '', content)

#remove some specific news/finance words that are killing it
stopwords2 = ['million', 'billion', 'millions', 'billions', 'said', 'xa', 'nthe']
def remove_stopwords2(content):
    cleaned = filter(lambda x: x not in stopwords2,content.split())
    return ' '.join(cleaned)

In [6]:
def preprocess(s):
    s=s.lower()
    s= raw_string(s)
    #stem/lemmatize
    
    #remove
    s= remove_punctuation(s)
    s= remove_stopwords(s)
    s= remove_stopwords2(s)
    s= remove_numbers(s)
    return s

from sklearn.feature_extraction.text import CountVectorizer
c_vectorizer = CountVectorizer(preprocessor=preprocess,
                               stop_words='english',
                               min_df=.05)

count_matrix = c_vectorizer.fit_transform(
    sql.read_sql("SELECT CleanedBody from ArticleCorpus",con=conn)['CleanedBody']
    )
n_topics = 5
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=10)
lda.fit(count_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [7]:
data = sql.read_sql("SELECT FullLink, CleanedBody from ArticleCorpus",con=conn)
results = lda.transform(c_vectorizer.fit_transform(data['CleanedBody']))
topic_df = pd.DataFrame(results,columns=["Topic%s" % i for i in range(5)])
topic_df['FullLink'] = data['FullLink']
topic_df.to_sql(name='Article_Topics',
            con=conn,
            if_exists='replace',
            index=False)
topic_df.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,FullLink
0,0.204758,0.202419,0.202136,176.188062,0.202625,http://citywire.co.uk/wealth-manager/news/satu...
1,40.186861,0.203943,0.202781,0.201028,0.205388,http://citywire.co.uk/wealth-manager/news/rath...
2,0.200000,0.200000,0.200000,0.200000,0.200000,http://citywire.co.uk/wealth-manager/news/cent...
3,0.201574,22.191722,0.202322,0.201939,0.202444,http://citywire.co.uk/wealth-manager/news/weal...
4,0.202711,0.203027,0.201879,0.202244,100.190139,http://citywire.co.uk/wealth-manager/news/jupi...
5,0.201377,94.194111,0.201508,0.201331,0.201672,http://citywire.co.uk/wealth-manager/news/towr...
6,113.194204,0.201158,0.201012,0.201810,0.201816,http://citywire.co.uk/wealth-manager/news/shel...
7,0.202878,75.186497,0.202547,0.205735,0.202343,http://citywire.co.uk/wealth-manager/news/hedg...
8,0.201800,89.188854,0.202262,0.202066,0.205019,http://citywire.co.uk/wealth-manager/news/spec...
9,0.203007,0.203070,0.201721,0.201011,37.191192,http://citywire.co.uk/wealth-manager/news/auro...


In [8]:
n_top_words = 15
import operator
vocabulary = c_vectorizer.get_feature_names()
for i in range(n_topics):
    best_words_indexes = lda.components_[i].argsort()[:-n_top_words - 1:-1]
    best_words = " ".join([vocabulary[i] for i in best_words_indexes])
    print best_words

oil month market price bank bull producers prices director wealth trading stocks research ftse head
fund management service investors advice retirement companies wealth income offer company period risk people mutual
jersey eu market uk business european opportunities client brexit result office management financial investors private
times brexit government vote companies following bank fund global emerging crisis holding asian largest profit
uk investment management manager share davies companies fund financial dividend office team trust year oil



Possible topics could be:
1. Investment Strategies
2. Wealth / D2C focused market
3. EU-centric Brexit topics
4. Macroeconomic concerns
5. Value/Equity investing


In [9]:
conn.close()